# Optimizers

In [1]:
import numpy as np

- SGD(Stochastic Gradient Descent): 확률적 경사 하강법

In [2]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

In [3]:
# f(x,y) = x*x + y*y + xy - 4x - 8y
def func(params):
    x, y = params['x'], params['y']
    return x*x + y*y + x*y - 4.*x - 8.*y

# Df(x,y) = (2x + y - 4, 2y + x - 8)
def deriv_f(params):
    x, y = params['x'], params['y']
    return {'x': round(2*x + y - 4., 4), 'y': round(2*y + x - 8., 4)}

In [4]:
sgd = SGD(0.5)
params = {'x':0., 'y':0.}
grads = deriv_f(params)
print(f'초기값: params={params}, grads={grads}, func={func(params):.4f}')
for i in range(10):
    sgd.update(params, grads)
    print(f'{i+1}회 시행: params={params}, grads={grads}, func={func(params):.4f}')
    grads = deriv_f(params)

초기값: params={'x': 0.0, 'y': 0.0}, grads={'x': -4.0, 'y': -8.0}, func=0.0000
1회 시행: params={'x': 2.0, 'y': 4.0}, grads={'x': -4.0, 'y': -8.0}, func=-12.0000
2회 시행: params={'x': 0.0, 'y': 3.0}, grads={'x': 4.0, 'y': 2.0}, func=-15.0000
3회 시행: params={'x': 0.5, 'y': 4.0}, grads={'x': -1.0, 'y': -2.0}, func=-15.7500
4회 시행: params={'x': 0.0, 'y': 3.75}, grads={'x': 1.0, 'y': 0.5}, func=-15.9375
5회 시행: params={'x': 0.125, 'y': 4.0}, grads={'x': -0.25, 'y': -0.5}, func=-15.9844
6회 시행: params={'x': 0.0, 'y': 3.9375}, grads={'x': 0.25, 'y': 0.125}, func=-15.9961
7회 시행: params={'x': 0.03125, 'y': 4.0}, grads={'x': -0.0625, 'y': -0.125}, func=-15.9990
8회 시행: params={'x': 0.0, 'y': 3.9844}, grads={'x': 0.0625, 'y': 0.0312}, func=-15.9998
9회 시행: params={'x': 0.0078, 'y': 4.0}, grads={'x': -0.0156, 'y': -0.0312}, func=-15.9999
10회 시행: params={'x': 0.0, 'y': 3.9961}, grads={'x': 0.0156, 'y': 0.0078}, func=-16.0000


- Momentum
    - Gradient Descent에 현재의 관성을 추가

In [5]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

In [6]:
momentum = Momentum(lr=0.5, momentum=0.5)
params = {'x':0., 'y':0.}
grads = deriv_f(params)
print(f'초기값: params={params}, grads={grads}, func={func(params):.4f}')
for i in range(10):
    momentum.update(params, grads)
    print(f'{i+1}회 시행: params={params}, grads={grads}, v={momentum.v}, func={func(params):.4f}')
    grads = deriv_f(params)

초기값: params={'x': 0.0, 'y': 0.0}, grads={'x': -4.0, 'y': -8.0}, func=0.0000
1회 시행: params={'x': 2.0, 'y': 4.0}, grads={'x': -4.0, 'y': -8.0}, v={'x': 2.0, 'y': 4.0}, func=-12.0000
2회 시행: params={'x': 1.0, 'y': 5.0}, grads={'x': 4.0, 'y': 2.0}, v={'x': -1.0, 'y': 1.0}, func=-13.0000
3회 시행: params={'x': -1.0, 'y': 4.0}, grads={'x': 3.0, 'y': 3.0}, v={'x': -2.0, 'y': -1.0}, func=-15.0000
4회 시행: params={'x': -1.0, 'y': 4.0}, grads={'x': -2.0, 'y': -1.0}, v={'x': 0.0, 'y': 0.0}, func=-15.0000
5회 시행: params={'x': 0.0, 'y': 4.5}, grads={'x': -2.0, 'y': -1.0}, v={'x': 1.0, 'y': 0.5}, func=-15.7500
6회 시행: params={'x': 0.25, 'y': 4.25}, grads={'x': 0.5, 'y': 1.0}, v={'x': 0.25, 'y': -0.25}, func=-15.8125
7회 시행: params={'x': 0.0, 'y': 3.75}, grads={'x': 0.75, 'y': 0.75}, v={'x': -0.25, 'y': -0.5}, func=-15.9375
8회 시행: params={'x': 0.0, 'y': 3.75}, grads={'x': -0.25, 'y': -0.5}, v={'x': 0.0, 'y': 0.0}, func=-15.9375
9회 시행: params={'x': 0.125, 'y': 4.0}, grads={'x': -0.25, 'y': -0.5}, v={'x': 0.125

- NAG(Nesterov Accelerated Gradient)
    * 현재 위치에서의 관성과 관성방향으로 움직인 후 위치에서의 gradient 반대방향을 합침

In [16]:
class Nesterov:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            params[key] += self.momentum * self.momentum * self.v[key]
            params[key] -= (1 + self.momentum) * self.lr * grads[key]
            self.v[key] *= self.momentum
            self.v[key] -= self.lr * grads[key]

In [19]:
nesterov = Nesterov(lr=0.2, momentum=0.8)
params = {'x':0., 'y':0.}
grads = deriv_f(params)
print(f'초기값: params={params}, grads={grads}, func={func(params):.4f}')
for i in range(10):
    nesterov.update(params, grads)
    print(f'{i+1}회 시행: params={params}, grads={grads}, v={momentum.v}, func={func(params):.4f}')
    grads = deriv_f(params)

초기값: params={'x': 0.0, 'y': 0.0}, grads={'x': -4.0, 'y': -8.0}, func=0.0000
1회 시행: params={'x': 1.4400000000000002, 'y': 2.8800000000000003}, grads={'x': -4.0, 'y': -8.0}, v={'x': -0.0625, 'y': 0.0625}, func=-14.2848
2회 시행: params={'x': 1.3184000000000005, 'y': 4.192000000000001}, grads={'x': 1.76, 'y': -0.8}, v={'x': -0.0625, 'y': 0.0625}, func=-13.9718
3회 시행: params={'x': 0.48435200000000034, 'y': 4.500736000000002}, grads={'x': 2.8288, 'y': 1.7024}, v={'x': -0.0625, 'y': 0.0625}, func=-15.2721
4회 시행: params={'x': -0.2592623999999998, 'y': 4.485220800000002}, grads={'x': 1.4694, 'y': 1.4858}, v={'x': -0.0625, 'y': 0.0625}, func=-15.8231
5회 시행: params={'x': -0.6070619199999999, 'y': 4.454504640000002}, grads={'x': -0.0333, 'y': 0.7112}, v={'x': -0.0625, 'y': 0.0625}, func=-15.7008
6회 시행: params={'x': -0.617173536, 'y': 4.435039712000003}, grads={'x': -0.7596, 'y': 0.3019}, v={'x': -0.0625, 'y': 0.0625}, func=-15.6983
7회 시행: params={'x': -0.4590508288, 'y': 4.376727769600003}, grads={'

- AdaGrad
    - 일정한 learning rate를 사용하지 않고 변수마다 그리고 스텝마다 learning rate가 바뀜

In [57]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key in params.keys():
                self.h[key] = 0.
                
        for key in params.keys():
            self.h[key] = round(self.h[key] + grads[key] * grads[key], 4)
            params[key] = round(params[key] - self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7), 4)

In [58]:
adg = AdaGrad(lr=10)
params = {'x':0., 'y':0.}
grads = deriv_f(params)
print(f'초기값: params={params}, grads={grads}, func={func(params):.4f}')
for i in range(10):
    adg.update(params, grads)
    print(f'{i+1}회 시행: params={params}, grads={grads}, h={adg.h}, func={func(params):.4f}')
    grads = deriv_f(params)

초기값: params={'x': 0.0, 'y': 0.0}, grads={'x': -4.0, 'y': -8.0}, func=0.0000
1회 시행: params={'x': 10.0, 'y': 10.0}, grads={'x': -4.0, 'y': -8.0}, h={'x': 16.0, 'y': 64.0}, func=180.0000
2회 시행: params={'x': 0.1163, 'y': 0.6021}, grads={'x': 26.0, 'y': 22.0}, h={'x': 692.0, 'y': 548.0}, func=-4.8359
3회 시행: params={'x': 1.3109, 'y': 3.3459}, grads={'x': -3.1653, 'y': -6.6795}, h={'x': 702.0191, 'y': 592.6157}, func=-14.7112
4회 시행: params={'x': 0.5703, 'y': 3.3448}, grads={'x': 1.9677, 'y': 0.0027}, h={'x': 705.8909, 'y': 592.6157}, func=-15.6191
5회 시행: params={'x': 0.3876, 'y': 3.6487}, grads={'x': 0.4854, 'y': -0.7401}, h={'x': 706.1265, 'y': 593.1634}, func=-15.8625
6회 시행: params={'x': 0.2281, 'y': 3.778}, grads={'x': 0.4239, 'y': -0.315}, h={'x': 706.3062, 'y': 593.2626}, func=-15.9493
7회 시행: params={'x': 0.14, 'y': 3.8666}, grads={'x': 0.2342, 'y': -0.2159}, h={'x': 706.361, 'y': 593.3092}, func=-15.9813
8회 시행: params={'x': 0.0848, 'y': 3.9187}, grads={'x': 0.1466, 'y': -0.1268}, h={'x'

- RMSProp
    - AdaGrad는 스텝이 많이 진행되면 h 값이 너무 커져서 학습률이 너무 작아져 학습이 거의 되지 않음
    - 이를 보완하기 위해 이전 누적치와 현재 그래디언트의 좌표별 제곱의 가중치 평균을 반영함

In [59]:
class RMSProp:
    def __init__(self, lr=0.01, gamma=0.75):    # gamma: forgetting factor(decay rate)
        self.lr = lr
        self.gamma = gamma      # gamma가 클수록 과거가 중요하고, 작을수록 현재(gradient)가 중요
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key in params.keys():
                self.h[key] = 0.
                
        for key in params.keys():
            self.h[key] = round(self.gamma * self.h[key] + (1 - self.gamma) * grads[key] * grads[key], 4)
            params[key] = round(params[key] - self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7), 4)

In [60]:
rmsp = RMSProp(lr=0.9, gamma=0.75)
params = {'x':0., 'y':0.}
grads = deriv_f(params)
print(f'초기값: params={params}, grads={grads}, func={func(params):.4f}')
for i in range(10):
    rmsp.update(params, grads)
    print(f'{i+1}회 시행: params={params}, grads={grads}, h={rmsp.h}, func={func(params):.4f}')
    grads = deriv_f(params)

초기값: params={'x': 0.0, 'y': 0.0}, grads={'x': -4.0, 'y': -8.0}, func=0.0000
1회 시행: params={'x': 1.8, 'y': 1.8}, grads={'x': -4.0, 'y': -8.0}, h={'x': 4.0, 'y': 16.0}, func=-11.8800
2회 시행: params={'x': 1.1255, 'y': 2.4324}, grads={'x': 1.4, 'y': -2.6}, h={'x': 3.49, 'y': 13.69}, func=-14.0402
3회 시행: params={'x': 0.7535, 'y': 2.971}, grads={'x': 0.6834, 'y': -2.0097}, h={'x': 2.7343, 'y': 11.2772}, func=-15.1487
4회 시행: params={'x': 0.4572, 'y': 3.3649}, grads={'x': 0.478, 'y': -1.3045}, h={'x': 2.1078, 'y': 8.8833}, func=-15.6780
5회 시행: params={'x': 0.2585, 'y': 3.6449}, grads={'x': 0.2793, 'y': -0.813}, h={'x': 1.6004, 'y': 6.8277}, func=-15.8989
6회 시행: params={'x': 0.1259, 'y': 3.8237}, grads={'x': 0.1619, 'y': -0.4517}, h={'x': 1.2069, 'y': 5.1718}, func=-15.9753
7회 시행: params={'x': 0.0545, 'y': 3.9271}, grads={'x': 0.0755, 'y': -0.2267}, h={'x': 0.9066, 'y': 3.8917}, func=-15.9957
8회 시행: params={'x': 0.0151, 'y': 3.9752}, grads={'x': 0.0361, 'y': -0.0913}, h={'x': 0.6803, 'y': 2.9209

- Adam
    - Momentum과 RMSProp 두가지 방식을 혼합

In [70]:
class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr, self.beta1, self.beta2 = lr, beta1, beta2
        self.iter, self.m, self.v = 0, None, None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key in params.keys():
                self.m[key], self.v[key] = 0., 0.
                
        self.iter += 1
        lr_t = self.lr * np.sqrt(1. - self.beta2**self.iter) / (1. - self.beta1**self.iter)
        
        for key in params.keys():
            self.m[key] = round(self.beta1 * self.m[key] + (1. - self.beta1) * grads[key], 4)
            # self.m[key] += (1. - self.beta1) * (grads[key] - self.m[key])
            self.v[key] = round(self.beta2 * self.v[key] + (1. - self.beta2) * grads[key]**2, 4)
            # self.v[key] += (1. - self.beta2) * (grads[key]**2 - self.v[key])
            params[key] = round(params[key] - lr_t * self.m[key] / (np.sqrt(self.v[key] + 1e-7)), 4)

In [71]:
adam = Adam(lr=0.9)
params = {'x':0., 'y':0.}
grads = deriv_f(params)
print(f'초기값: params={params}, grads={grads}, func={func(params):.4f}')
for i in range(10):
    adam.update(params, grads)
    print(f'{i+1}회 시행: params={params}, grads={grads}, m={adam.m}, v={adam.v}, func={func(params):.4f}')
    grads = deriv_f(params)

초기값: params={'x': 0.0, 'y': 0.0}, grads={'x': -4.0, 'y': -8.0}, func=0.0000
1회 시행: params={'x': 0.9, 'y': 0.9}, grads={'x': -4.0, 'y': -8.0}, m={'x': -0.4, 'y': -0.8}, v={'x': 0.016, 'y': 0.064}, func=-8.3700
2회 시행: params={'x': 1.68, 'y': 1.7728}, grads={'x': -1.3, 'y': -5.3}, m={'x': -0.49, 'y': -1.25}, v={'x': 0.0177, 'y': 0.092}, func=-11.9589
3회 시행: params={'x': 2.1122, 'y': 2.5807}, grads={'x': 1.1328, 'y': -2.7744}, m={'x': -0.3277, 'y': -1.4024}, v={'x': 0.019, 'y': 0.0996}, func=-12.5220
4회 시행: params={'x': 2.1267, 'y': 3.2788}, grads={'x': 2.8051, 'y': -0.7264}, m={'x': -0.0144, 'y': -1.3348}, v={'x': 0.0268, 'y': 0.1}, func=-12.4908
5회 시행: params={'x': 1.8599, 'y': 3.8339}, grads={'x': 3.5322, 'y': 0.6843}, m={'x': 0.3403, 'y': -1.1329}, v={'x': 0.0392, 'y': 0.1004}, func=-12.8221
6회 시행: params={'x': 1.4279, 'y': 4.236}, grads={'x': 3.5537, 'y': 1.5277}, m={'x': 0.6616, 'y': -0.8668}, v={'x': 0.0518, 'y': 0.1026}, func=-13.5684
7회 시행: params={'x': 0.9013, 'y': 4.4971}, grads